# 指定パターン数を出力する

In [53]:
import random
import json
import lzma

COLORS_LIST = ['赤','白','黄','桃','紫']
COLORS_SET = set(COLORS_LIST)
ALL_PATTERN = [f"{a} {b} {c} {d} {e}" for a in COLORS_LIST for b in COLORS_LIST for c in COLORS_LIST for d in COLORS_LIST for e in COLORS_LIST]
print("len(ALL_PATTERN)=",len(ALL_PATTERN))
COLORS_COUNT_OF_SAMPLES = 999

len(ALL_PATTERN)= 3125


In [54]:
def get_random_colors(not_list=[]):
    enables = list(set(ALL_PATTERN)-set(not_list))
    return random.choice(enables)
    
get_random_colors()

'黄 白 赤 白 赤'

In [55]:
def tulip_x5(input_str):
    output_list = []
    input_set1 = set(input_str.split())
    if len(input_set1) == 1:
        output_list.append(f'全部{input_set1.pop()}！')
    elif len(input_set1) == 5:
        output_list.append('キレイだなぁ〜')
    else:
        output_list.append('{}が欲しいなぁ〜'.format("と".join(list(COLORS_SET - input_set1))))
    return output_list

random_colors = get_random_colors()
output_list = tulip_x5(random_colors)
print(random_colors, output_list)

白 白 桃 白 桃 ['紫と黄と赤が欲しいなぁ〜']


In [56]:
def output_json_xz(xz_filename, json_data):
    # JSONデータを整形して文字列として取得
    # ensure_ascii=False すると日本語がエスケープされなくなるためjsonの可読性はあがりますが互換性は低くなります。注意。
    json_str = json.dumps(json_data, indent=4, sort_keys=True, ensure_ascii=False)

    # JSONファイルをXZ形式で圧縮する
    with lzma.open(xz_filename, 'wb') as xz_out:
        xz_out.write(json_str.encode('utf-8'))

In [80]:
def output_all(out_xz_filename, pattern):
    pattern_keta = len(str(pattern)) # 桁数を数える（文字列に変換後、文字列長で取得できる）
    output_index = 0
    all_inputs = []
    json_data = {}

    # 全部同じ色が出るケースは稀にしか存在しないので予め作成
    all_same_color_count = max(1+pattern//200, len(COLORS_LIST)) # 200個に一個は全部同色の文字列を入れる
    all_same_color_list = COLORS_LIST.copy()
    random.shuffle(all_same_color_list)
    for answer_id in range(1,all_same_color_count+1):
        input_list = []

        # チューリップｘ５の解答を求める
        color = all_same_color_list[answer_id-1]
        input_str = f"{color} {color} {color} {color} {color}"
        input_list.append(input_str)
        all_inputs.append(input_str)
        output_list = tulip_x5(input_str)

        # 入力と出力をJSONに詰める
        answer_key = f"answer{answer_id:0{pattern_keta}}"
        json_data[answer_key] = [input_list, output_list]
        
    # 残りはランダムで入れる
    for answer_id in range(all_same_color_count+1, pattern+1):
        input_list = []

        # randomで入力文字列を取得する
        input_str = get_random_colors(all_inputs)
        input_list.append(input_str)
        all_inputs.append(input_str)

        # チューリップｘ５の解答を求める
        output_list = tulip_x5(input_str)

        # 入力と出力をJSONに詰める
        answer_key = f"answer{answer_id:0{pattern_keta}}"
        json_data[answer_key] = [input_list, output_list]

    # jsonファイルをxz出力する
    output_json_xz(out_xz_filename, json_data)

    # 重複がなければpatternと一致するはず
    print('all_inputs count=',len(set(all_inputs)))

In [81]:
output_all("data/answer.json.xz", COLORS_COUNT_OF_SAMPLES)

all_inputs count= 999


#### 赤と紫が欲しいなぁ / 紫と赤が欲しいなぁ など、組み合わせの表示順が違うケースを考慮して正解判定を行う

In [89]:
from collections import Counter

def tulip_judge_answer_colors(answer_text, input_text):
    pos1 = answer_text.find("が欲しいなぁ")
    pos2 = input_text.find("が欲しいなぁ")
    if pos1 >= 0 and pos2 >= 0 and pos1 == pos2:
        answers = answer_text[:pos1].split("と")
        inputs = input_text[:pos2].split("と")
        print(Counter(answers))
        print(Counter(inputs))
        return Counter(answers) == Counter(inputs)
    return False

#tulip_judge_answer_colors('赤と紫が欲しいなぁ','紫と赤が欲しいなぁ')
#tulip_judge_answer_colors('赤が欲しいなぁ','紫と赤が欲しいなぁ')
#tulip_judge_answer_colors('赤と紫が欲しいなぁ','紫と赤と黄が欲しいなぁ')
tulip_judge_answer_colors('黄と赤と紫が欲しいなぁ','紫と赤と黄が欲しいなぁ')

Counter({'黄': 1, '赤': 1, '紫': 1})
Counter({'紫': 1, '赤': 1, '黄': 1})


True